# Model Training – 1D CNN (Skeleton)
This notebook turns OTDR traces into windowed examples and trains a tiny 1D CNN to classify *event* vs *normal* windows.


In [5]:
# ---- make imports robust no matter where the notebook is run ----
import sys, os
from pathlib import Path

here = Path.cwd()
root = here
for _ in range(4):  # coba naik max 4 level
    if (root / "src" / "otdr_parser.py").exists():
        break
    root = root.parent

init_file = Path(root) / "src" / "__init__.py"
if not init_file.exists():
    init_file.touch()   # hanya bikin kalau belum ada

from src.otdr_parser import load_trace, smooth_power
print("Project root:", root)


Project root: /mnt/d/Projects/fiber-otdr-ml


In [7]:
import pandas as pd, numpy as np, tensorflow as tf
from src.otdr_parser import load_trace, smooth_power

df = load_trace(str(root / 'sample_data' / 'trace_sample.csv'))
x = df['distance_km'].to_numpy(); y = smooth_power(df['power_db'].to_numpy(), window=7)
window = 8
# Build supervised windows (toy labels: mark windows crossing distance>=5.0 as event)
X, y_label = [], []
for i in range(len(y)-window):
    X.append(y[i:i+window])
    y_label.append(1 if x[i+window-1] >= 5.0 else 0)
X = np.array(X)[..., None]
y_label = np.array(y_label)
print(X.shape, y_label.mean())


(5, 8, 1) 0.4


In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=(X.shape[1],1)),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X, y_label, epochs=10, validation_split=0.2, verbose=0)
print('val acc:', history.history['val_accuracy'][-1])


/home/milzon/.venvs/ai/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-09-20 09:55:35.740026: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


val acc: 0.0
